# Read this in conjuction with my post on medium
##

In [1]:
# We will need whole bunch of libraries
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import multiprocessing
from multiprocessing import Pool
from multiprocessing import cpu_count
import gensim
from gensim.models.doc2vec import TaggedDocument
import os
import collections
import smart_open
import random

# Pre-processing and Tokenization of data

In [2]:
#Load file from modified raw data set
df=pd.read_pickle("uniform.pkl")
# Check the size of the dataset
df.shape

(1619165, 11)

In [3]:
# We have created a file prep2 containing functions to be used in multiprocessing code below
# This is because of a bug in Jupyter that cause code to be stuck in an infinite loop, if these functions are build here.
import prep2
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
%config IPCompleter.greedy=True
# This part was needed as any changes to the function in prep2.py is not reflected without this.
import importlib
importlib.reload(prep2)
import warnings
warnings.filterwarnings('ignore')

In [4]:
# We will use this a document Id for Doc2vec tagging
df['token']=  df["index"].map(str)+"_" + df["overall"].map(str)
df['token'].head()

0    2880115_1.0
1    1300387_1.0
2    4487515_1.0
3    6588495_1.0
4    3653311_1.0
Name: token, dtype: object

In [5]:
# Some useful references on this:
# https://praveenbezawada.com/2018/01/25/document-similarity-using-gensim-dec2vec/
# https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/doc2vec-lee.ipynb
# https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/doc2vec-IMDB.ipynb

In [6]:
# Prepocessing and tokenizing the reviews.
docs=df[['reviewText','token']]
docs['TokenizedText']=prep2.parallelize(docs['reviewText'], prep2.word_func)

# Building and Training Doc2vec models

In [7]:
# Tokenized text are list in each cell of dataframe. 
# This is doc2vec software needs to convert elements in a column in a list and attach a tag to it
# This was borrowed from here: https://praveenbezawada.com/2018/01/25/document-similarity-using-gensim-dec2vec/

# Tagging the documents
class TaggedDocumentIterator(object):
    def __init__(self, doc_list, labels_list):
        self.labels_list = labels_list
        self.doc_list = doc_list
    def __iter__(self):
        for idx, doc in enumerate(self.doc_list):
            yield TaggedDocument(words=doc, tags=[self.labels_list[idx]])
 
docLabels = list(docs['token'])
data = list(docs['TokenizedText'])
sentences = TaggedDocumentIterator(data, docLabels)

In [8]:
# This gives use two models one for DM and other for DBOW
%time simple_models = prep2.doc_model(sentences)

Doc2Vec(dbow,d300,n5,mc2,t32) vocabulary scanned & state initialized
Doc2Vec("alpha=0.05",dm/m,d300,n5,w10,mc2,t32) vocabulary scanned & state initialized
Wall time: 5min 48s


In [ ]:
# Next two steps is training the models
# This is DBOW
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
print("Training %s" % simple_models[0])
simple_models[0].train(sentences, total_examples=simple_models[0].corpus_count , epochs=15)

In [ ]:
# This is DM
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
print("Training %s" % simple_models[1])
simple_models[1].train(sentences, total_examples=simple_models[1].corpus_count , epochs=15)

In [ ]:
simple_models[0].save('DBOW.doc2vec')
simple_models[1].save('DM.doc2vec')